In [317]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [318]:
df = pd.read_csv('train.csv')
df = df.drop('id',axis=1)
y = df['target']
X = df.drop('target',axis=1)
test = pd.read_csv('test.csv')
test.drop('id',axis=1,inplace=True)
X_mean = X.mean(axis=1)
X['mean']  = X_mean
test_mean = test.mean(axis=1)
test['mean'] = test_mean

In [339]:
features = [ '16', '33', '43', '45', '52', '63', '65', '73', '90', '91', '117', '133', '134', '149', '189', '199', '217', '237', '258', '295']
X1 = X[features]
test = test[features]
print(len(X1), len(test))

250 19750


In [302]:
params = {}
params['random_state'] = 0
params['n_jobs'] = -1
params['C'] = 0.126
params['penalty'] = 'l1'
params['class_weight'] = 'balance'
params['solver'] = 'saga'
best = params

In [303]:
def train(X1, y, best):
    folds = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 20, random_state = 0)
    oof =  np.zeros(len(X1))
    clfs = []
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X1, y)):
            #print('--- Fold {} started at {}'.format(n_fold, time.ctime()))

            train_x, train_y = X1.iloc[train_idx], y.iloc[train_idx]
            valid_x, valid_y = X1.iloc[valid_idx], y.iloc[valid_idx]

            clf = LogisticRegression(**best)
            clf.fit(train_x, train_y)
            clfs.append(clf)
            
            oof[valid_idx] += clf.predict_proba(valid_x)[:,1]
            
    return clfs, oof

In [304]:
def predict(clfs,test):
    predictions = np.zeros(len(test))
    for c in clfs:
        predictions += c.predict_proba(test)[:,1]
    predictions = predictions/len(clfs)
    return predictions

In [305]:
clfs, oof = train(X1 ,y, best)
roc_auc_score(y, oof/20)

0.8629166666666667

In [306]:
predicitons =  predict(clfs,test)

Submission

In [307]:
result = pd.read_csv('sample_submission (1).csv')
result['target'] = predicitons
result.to_csv('submission13.csv', index=False)

Keras

In [312]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import *

In [430]:
es_callback = keras.callbacks.EarlyStopping(monitor='auc', patience=80)

In [475]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation='relu', input_dim=len(features)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X1,y,batch_size=40, 
                    epochs=290, validation_split=0.25, shuffle=True, callbacks=[es_callback])

Train on 187 samples, validate on 63 samples
Epoch 1/290
187/187 [==============================] - 1s 6ms/step - loss: 0.7282 - acc: 0.5775 - val_loss: 0.7223 - val_acc: 0.4921
Epoch 2/290
187/187 [==============================] - 0s 37us/step - loss: 0.7446 - acc: 0.5722 - val_loss: 0.7178 - val_acc: 0.4921
Epoch 3/290
187/187 [==============================] - 0s 37us/step - loss: 0.6694 - acc: 0.5829 - val_loss: 0.7131 - val_acc: 0.4921
Epoch 4/290
187/187 [==============================] - 0s 37us/step - loss: 0.7346 - acc: 0.5989 - val_loss: 0.7089 - val_acc: 0.4921
Epoch 5/290
187/187 [==============================] - 0s 37us/step - loss: 0.7129 - acc: 0.6524 - val_loss: 0.7049 - val_acc: 0.5079
Epoch 6/290
187/187 [==============================] - 0s 37us/step - loss: 0.7328 - acc: 0.5775 - val_loss: 0.7005 - val_acc: 0.5079
Epoch 7/290
187/187 [==============================] - 0s 43us/step - loss: 0.7239 - acc: 0.6203 - val_loss: 0.6966 - val_acc: 0.5079
Epoch 8/290
187/18

C:\Users\kexu\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `auc` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


187/187 [==============================] - 0s 37us/step - loss: 0.6304 - acc: 0.6631 - val_loss: 0.6582 - val_acc: 0.6032
Epoch 27/290
187/187 [==============================] - 0s 43us/step - loss: 0.6267 - acc: 0.6738 - val_loss: 0.6567 - val_acc: 0.6032
Epoch 28/290
187/187 [==============================] - 0s 38us/step - loss: 0.6083 - acc: 0.6524 - val_loss: 0.6552 - val_acc: 0.6032
Epoch 29/290
187/187 [==============================] - 0s 43us/step - loss: 0.5928 - acc: 0.6791 - val_loss: 0.6540 - val_acc: 0.6032
Epoch 30/290
187/187 [==============================] - 0s 37us/step - loss: 0.5942 - acc: 0.6738 - val_loss: 0.6525 - val_acc: 0.6190
Epoch 31/290
187/187 [==============================] - 0s 32us/step - loss: 0.5968 - acc: 0.6898 - val_loss: 0.6510 - val_acc: 0.6190
Epoch 32/290
187/187 [==============================] - 0s 37us/step - loss: 0.6053 - acc: 0.6524 - val_loss: 0.6494 - val_acc: 0.6190
Epoch 33/290
187/187 [==============================] - 0s 43us/step

Epoch 87/290
187/187 [==============================] - 0s 37us/step - loss: 0.4188 - acc: 0.7754 - val_loss: 0.5486 - val_acc: 0.7619
Epoch 88/290
187/187 [==============================] - 0s 37us/step - loss: 0.4437 - acc: 0.7594 - val_loss: 0.5469 - val_acc: 0.7619
Epoch 89/290
187/187 [==============================] - 0s 37us/step - loss: 0.4507 - acc: 0.7968 - val_loss: 0.5455 - val_acc: 0.7619
Epoch 90/290
187/187 [==============================] - 0s 32us/step - loss: 0.4449 - acc: 0.7594 - val_loss: 0.5442 - val_acc: 0.7619
Epoch 91/290
187/187 [==============================] - 0s 37us/step - loss: 0.4795 - acc: 0.7487 - val_loss: 0.5422 - val_acc: 0.7619
Epoch 92/290
187/187 [==============================] - 0s 32us/step - loss: 0.4003 - acc: 0.7540 - val_loss: 0.5406 - val_acc: 0.7460
Epoch 93/290
187/187 [==============================] - 0s 37us/step - loss: 0.4409 - acc: 0.7380 - val_loss: 0.5392 - val_acc: 0.7460
Epoch 94/290
187/187 [==============================] -

187/187 [==============================] - 0s 37us/step - loss: 0.2954 - acc: 0.8396 - val_loss: 0.5166 - val_acc: 0.8095
Epoch 148/290
187/187 [==============================] - 0s 43us/step - loss: 0.2596 - acc: 0.8984 - val_loss: 0.5180 - val_acc: 0.8254
Epoch 149/290
187/187 [==============================] - 0s 37us/step - loss: 0.3147 - acc: 0.8610 - val_loss: 0.5190 - val_acc: 0.8254
Epoch 150/290
187/187 [==============================] - 0s 43us/step - loss: 0.2993 - acc: 0.8556 - val_loss: 0.5200 - val_acc: 0.8254
Epoch 151/290
187/187 [==============================] - 0s 43us/step - loss: 0.2834 - acc: 0.8663 - val_loss: 0.5228 - val_acc: 0.8254
Epoch 152/290
187/187 [==============================] - 0s 37us/step - loss: 0.2792 - acc: 0.9198 - val_loss: 0.5238 - val_acc: 0.8254
Epoch 153/290
187/187 [==============================] - 0s 43us/step - loss: 0.2568 - acc: 0.9144 - val_loss: 0.5238 - val_acc: 0.8254
Epoch 154/290
187/187 [==============================] - 0s 37

187/187 [==============================] - 0s 32us/step - loss: 0.2085 - acc: 0.9305 - val_loss: 0.6622 - val_acc: 0.8254
Epoch 208/290
187/187 [==============================] - 0s 32us/step - loss: 0.2518 - acc: 0.8556 - val_loss: 0.6661 - val_acc: 0.8254
Epoch 209/290
187/187 [==============================] - 0s 32us/step - loss: 0.2137 - acc: 0.9037 - val_loss: 0.6699 - val_acc: 0.8254
Epoch 210/290
187/187 [==============================] - 0s 37us/step - loss: 0.2328 - acc: 0.8930 - val_loss: 0.6740 - val_acc: 0.8254
Epoch 211/290
187/187 [==============================] - 0s 37us/step - loss: 0.2060 - acc: 0.9251 - val_loss: 0.6765 - val_acc: 0.8254
Epoch 212/290
187/187 [==============================] - 0s 37us/step - loss: 0.2334 - acc: 0.9037 - val_loss: 0.6803 - val_acc: 0.8254
Epoch 213/290
187/187 [==============================] - 0s 32us/step - loss: 0.2353 - acc: 0.8824 - val_loss: 0.6839 - val_acc: 0.8095
Epoch 214/290
187/187 [==============================] - 0s 37

187/187 [==============================] - 0s 32us/step - loss: 0.1663 - acc: 0.9412 - val_loss: 0.8686 - val_acc: 0.8095
Epoch 268/290
187/187 [==============================] - 0s 37us/step - loss: 0.1435 - acc: 0.9519 - val_loss: 0.8741 - val_acc: 0.7937
Epoch 269/290
187/187 [==============================] - 0s 32us/step - loss: 0.1606 - acc: 0.9412 - val_loss: 0.8787 - val_acc: 0.7937
Epoch 270/290
187/187 [==============================] - 0s 37us/step - loss: 0.1337 - acc: 0.9519 - val_loss: 0.8813 - val_acc: 0.7937
Epoch 271/290
187/187 [==============================] - 0s 32us/step - loss: 0.1021 - acc: 0.9786 - val_loss: 0.8829 - val_acc: 0.7937
Epoch 272/290
187/187 [==============================] - 0s 32us/step - loss: 0.1375 - acc: 0.9733 - val_loss: 0.8846 - val_acc: 0.7937
Epoch 273/290
187/187 [==============================] - 0s 32us/step - loss: 0.1160 - acc: 0.9840 - val_loss: 0.8867 - val_acc: 0.7937
Epoch 274/290
187/187 [==============================] - 0s 37

In [476]:
predictions = model.predict(test)

In [477]:
predictions

array([[0.99982077],
       [0.9287504 ],
       [0.9977241 ],
       ...,
       [0.521169  ],
       [1.        ],
       [0.00320656]], dtype=float32)

In [478]:
result = pd.read_csv('sample_submission (1).csv')
result['target'] = predictions
result.to_csv('submission11.csv', index = False)